In [7]:
%load_ext autoreload
%autoreload 2
import cf_ownership as cfo
import counterfeit_utils as cfu
from tqdm import tqdm
import matplotlib.pyplot as plt
import pickle


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
top_slugs = cfu.get_top_slugs(cfu.CUT_OFF,cfu.DB_NAME)
der_lists = cfu.der_list_from_db()


with open(f'top_owner_dfs/top_slugs.pkl','wb') as f:
    pickle.dump(top_slugs,f)
with open(f'top_owner_dfs/der_lists.pkl','wb') as f:
    pickle.dump(der_lists,f)



In [4]:
#count how many occurrences of each address
for top_slug in tqdm(top_slugs):
    df = cfu.get_slug_owners(top_slug)
    df.to_pickle(f'top_owner_dfs/{top_slug}_owners.pkl')
    # df_slugged = df.query('slug == @top_slug')
    # val_counts = df_slugged['address'].value_counts()
    # print(val_counts.describe())
    # #plot cdf of val counts
    # cdf = val_counts.value_counts(normalize=True).sort_index().cumsum()
    # cdf.plot()
    # plt.xlim(0,3)
    # plt.show()
    



100%|██████████| 54/54 [14:49<00:00, 16.46s/it]


In [ ]:

cfo.plot_all_overlap_cdfs()

0n1-force
['0n1-comics', '0n1-frame']
count    382395.000000
mean       4943.134408
std        3132.426148
min           1.000000
0.1%         16.000000
0.2%         16.000000
1%           67.000000
10%         737.000000
25%        2191.000000
50%        4703.000000
75%        7580.000000
max       10976.000000
dtype: float64
count    98386.000000
mean      5443.011465
std       3225.434427
min          1.000000
0.1%         9.000000
0.2%        16.000000
1%          79.000000
10%        973.000000
25%       2569.000000
50%       5502.000000
75%       8301.000000
max      10976.000000
dtype: float64
3landers
['']
count    1.413777e+06
mean     5.982562e+03
std      2.930274e+03
min      1.000000e+00
0.1%     1.700000e+01
0.2%     3.400000e+01
1%       1.360000e+02
10%      1.326000e+03
25%      3.802000e+03
50%      6.571000e+03
75%      8.327000e+03
max      1.097600e+04
dtype: float64
count    162027.000000
mean       4994.569282
std        3231.277425
min           1.000000
0.1%   

In [ ]:
all_owner_df = cfu.get_all_owners()

In [ ]:
all_owner_df.to_pickle('all_owner_df.pkl')